In [1]:
import os

In [2]:
%pwd

'c:\\Projects\\End-to-End-chest-classification-using-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Projects\\End-to-End-chest-classification-using-DVC'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/chetankharkar21/End-to-End-chest-classification-using-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="chetankharkar21"
os.environ["MLFLOW_TRACKING_PASSWORD"]="4d6584b697c03a8d1f682a054efbee4a7264d047"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data: Path
    all_params:dict
    mlflow_uri : str
    params_image_size:list
    params_batch_size:int

In [9]:
from cnnClassifier.constants import*
from cnnClassifier.utils.common import read_yaml,create_directories,save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        # Ensure read_yaml is correctly implemented and imported
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create root directory for artifacts
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data= "artifacts/data_ingestion",
            mlflow_uri="https://dagshub.com/chetankharkar21/End-to-End-chest-classification-using-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE

        )
        return eval_config
    


In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    def _valid_generator(self):
        datagenrator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenrator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod 
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the model registry, which depends on the use case
                # Please refer to the doc for further information
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-12-28 21:55:12,152: INFO: common: YAML file: config\config.yaml loaded successfully]
[2024-12-28 21:55:12,166: INFO: common: YAML file: params.yaml loaded successfully]
[2024-12-28 21:55:12,166: INFO: common: Created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 13s 2s/step - loss: 0.6106 - accuracy: 0.7647
[2024-12-28 21:55:25,886: INFO: common: JSON file saved at: scores.json]


2024/12/28 21:55:27 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-12-28 21:55:29,881: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 16). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Rutu\AppData\Local\Temp\tmp35z3k080\model\data\model\assets
[2024-12-28 21:55:31,479: INFO: builder_impl: Assets written to: C:\Users\Rutu\AppData\Local\Temp\tmp35z3k080\model\data\model\assets]


c:\Projects\End-to-End-chest-classification-using-DVC\cancer\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/12/28 21:56:25 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
